In [ ]:
from tweepy import Stream
import nltk
import pandas as pd
import re

import sqlite3
import sqlalchemy

In [ ]:
def clean_tweet(tweet):
    tweet = str(tweet)
    tweet = ' '.join([w for w in tweet.split() if len(w) < 20]) #remove crypto addresses from strings
    tweet = ' '.join(re.sub("(#[A-Za-z0-9]+)", " ", tweet).split()) #remove hashtags
    tweet = ' '.join(re.sub("(&[A-Za-z0-9]+)", " ", tweet).split()) #remove hwords starting with &
    tweet = ' '.join(re.sub("(https?:?\/\/\S+)", " ", tweet).split()) #remove hyperlinks
    tweet = ' '.join(re.sub("(rt @[A-Za-z0-9]+)|(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split()) #remove RT and mentions
    tweet = ' '.join(re.sub("\S*\d\S*", " ", tweet).split()) #remove words containing numbers
    tweet = ' '.join(re.sub("([0-9])", " ", tweet).split()) #remove numbers
    
    return tweet.lower()

In [ ]:
connection = sqlite3.connect('Crypto_Tweets.db')
engine = sqlalchemy.create_engine('sqlite:///Crypto_Tweets.db')

In [ ]:
access_token = 'xxxxxxx'
access_token_secret = 'xxxxxxxx'
consumer_key = 'xxxxxxxx'
consumer_secret = 'xxxxxxxx'

In [ ]:
used_id = {0}
class MyStreamListener(Stream):
    def on_status(self, status):
        if status.text.startswith('RT') or len(clean_tweet(status.text).split()) <= 3 or status.id in used_id:
            pass
        else:
            try:
                if hasattr(status, 'retweeted_status') and hasattr(status.retweeted_status, 'extended_tweet'):
                    #print('retweeted: ' + status.retweeted_status.extended_tweet['full_text'])
                    tweet_text = status.retweeted_status.extended_tweet['full_text']
                if hasattr(status, 'extended_tweet'):
                    #print('extended_tweet: ' + status.extended_tweet['full_text'])
                    tweet_text = status.extended_tweet['full_text']
                else:
                    #print('text: ' + status.text)
                    tweet_text = status.text
                    
                
                used_id.add(status.id)
                #print(status.id)
                #print(status.created_at)
                #print(clean_tweet(tweet_text))
                #print('\n')
                
                df = (pd.DataFrame([{'created_at': status.created_at, 
                                     'tweet_id': status.id,
                                     'text': status.text,
                                     'processed_text': clean_tweet(status.text)}])
                     )
                df.to_sql('Tweets', engine, if_exists = 'append', index = False)

            except AttributeError:
                print('attribute error: ' + status.text)

    def on_error(self, status_code):
        if status_code == 420:  # end of monthly limit rate (500k)
            return False


stream = MyStreamListener(access_token = access_token,
                          access_token_secret = access_token_secret,
                          consumer_key = consumer_key,
                          consumer_secret = consumer_secret)

stream.filter(track=['bitcoin', 'ethereum'], languages=["en"])